# Hardware Performance Testing

Test TinyMPC hardware performance by measuring execution time as a function of max_iter for different bitstreams

## 1. Setup and Imports

In [17]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import time
from pathlib import Path
import pandas as pd
from scipy import stats

# Add driver path
os.chdir("/home/xilinx/jupyter_notebooks/zhenyu/tinympc_ip_gen/")

sys.path.append('driver')
from tinympc_hw import tinympc_hw

# Import dynamics for test problem setup
from dynamics import LinearizedQuadcopterDynamics, CrazyflieParams, NoiseModel

print("All modules imported successfully")

All modules imported successfully


In [18]:
# Initialize dynamics model for test problem
params = CrazyflieParams()
noise_model = NoiseModel()
dynamics = LinearizedQuadcopterDynamics(params, noise_model)

# Generate system matrices
control_freq = 100.0  # Hz
A, B = dynamics.generate_system_matrices(control_freq)
Q, R = dynamics.generate_cost_matrices()
constraints = dynamics.generate_constraints()

# System dimensions (fixed for quadrotor)
nx = 12  # State dimension
nu = 4   # Control dimension

print(f"Test problem configured:")
print(f"  State dimension (nx): {nx}")
print(f"  Control dimension (nu): {nu}")
print(f"  Note: Prediction horizon (N) will be extracted from each bitstream")

Test problem configured:
  State dimension (nx): 12
  Control dimension (nu): 4
  Note: Prediction horizon (N) will be extracted from each bitstream


In [19]:
# Generate test data
def generate_test_data(nx, nu, N):
    """Generate random test data for MPC problem"""
    np.random.seed(42)  # For reproducibility
    
    # Initial state with some deviation from origin
    x0 = np.random.randn(nx) * 0.1
    x0[2] = 1.0  # Set altitude to 1m
    
    # Reference trajectory (hover at origin)
    xref = np.zeros((N, nx))
    xref[:, 2] = 1.0  # Reference altitude
    
    # Reference control (hover)
    uref = np.zeros((N-1, nu))
    
    return x0, xref, uref

def extract_N_from_bitstream(bitstream_path):
    """Extract N parameter from bitstream filename"""
    import re
    filename = bitstream_path.name if hasattr(bitstream_path, 'name') else str(bitstream_path)
    pattern = r'tinympcproj_N(\d+)_'
    match = re.search(pattern, filename)
    if match:
        return int(match.group(1))
    else:
        print(f"Warning: Could not extract N from {filename}, using default N=5")
        return 5

# Note: Test data will be generated per bitstream with correct N
print("Test data generation functions defined")

Test data generation functions defined


In [20]:
# Find all bitstream files in subdirectories
import os
from pathlib import Path
import glob

def find_all_bitstreams(base_path="."):
    """Find all .bit files in the project directory and subdirectories"""
    bitstream_files = []
    
    # Search for .bit files recursively
    search_pattern = os.path.join(base_path, "**", "*.bit")
    found_files = glob.glob(search_pattern, recursive=True)
    
    # Convert to Path objects and filter out any invalid paths
    for file_path in found_files:
        path_obj = Path(file_path)
        if path_obj.exists() and path_obj.is_file():
            bitstream_files.append(path_obj)
    
    # Also check specific known locations
    known_dirs = ["bitstream", "impl", "output", "build"]
    for dir_name in known_dirs:
        dir_path = Path(base_path) / dir_name
        if dir_path.exists() and dir_path.is_dir():
            bit_files = list(dir_path.glob("*.bit"))
            for bit_file in bit_files:
                if bit_file not in bitstream_files:
                    bitstream_files.append(bit_file)
    
    return sorted(bitstream_files)

def test_bitstream_performance(bitstream_path, test_maxiter_values=[10, 100, 1000], num_trials=10):
    """
    Test a bitstream with specific max_iter values, with warmup run
    
    Args:
        bitstream_path: Path to bitstream file
        test_maxiter_values: List of max_iter values to test (default: [10, 100, 1000])
        num_trials: Number of trials per max_iter value (excluding warmup)
    
    Returns:
        dict: Results with average execution times
    """
    # Extract N from bitstream filename
    N = extract_N_from_bitstream(bitstream_path)
    
    # Generate test data with correct dimensions
    nx = 12  # State dimension (fixed for quadrotor)
    nu = 4   # Control dimension (fixed for quadrotor)
    x0_test, xref_test, uref_test = generate_test_data(nx, nu, N)
    
    # Initialize hardware solver
    hw_solver = tinympc_hw(bitstream_path=str(bitstream_path))
    
    results = {
        'bitstream': str(bitstream_path),
        'N': N,
        'maxiter_values': test_maxiter_values,
        'avg_times': {},
        'all_times': {}
    }
    
    for max_iter in test_maxiter_values:
        # Set check_termination equal to max_iter as requested
        check_termination = max_iter
        hw_solver.setup(max_iter=max_iter, check_termination=check_termination, verbose=0)
        
        # Warmup run (first run to prepare hardware)
        hw_solver.set_x0(x0_test)
        hw_solver.set_x_ref(xref_test)
        hw_solver.set_u_ref(uref_test)
        hw_solver.solve(timeout=1.0)  # Warmup - don't record this time
        
        # Actual timing runs
        times = []
        for trial in range(num_trials):
            # Set problem data
            hw_solver.set_x0(x0_test)
            hw_solver.set_x_ref(xref_test)
            hw_solver.set_u_ref(uref_test)
            
            # Solve and get hardware execution time
            success = hw_solver.solve(timeout=1.0)
            
            if success:
                # Use the solver's internal timing (already in ms)
                exec_time = hw_solver.solve_time
                times.append(exec_time)
        
        if len(times) > 0:
            results['avg_times'][max_iter] = np.mean(times)
            results['all_times'][max_iter] = times
    
    # Cleanup
    hw_solver.cleanup()
    
    return results

# Find all available bitstreams
print("Searching for bitstream files...")
bitstreams = find_all_bitstreams()

if len(bitstreams) == 0:
    print("No bitstream files found. Looking in current directory and subdirectories...")
    # Try with absolute path
    bitstreams = find_all_bitstreams("/home/xilinx/jupyter_notebooks/zhenyu/tinympc_ip_gen/")

print(f"\nFound {len(bitstreams)} bitstream file(s):")
for idx, bitstream in enumerate(bitstreams):
    print(f"  {idx+1}. {bitstream}")

# Select first bitstream as default if available
if len(bitstreams) > 0:
    selected_bitstream = bitstreams[0]
    print(f"\nDefault selected bitstream: {selected_bitstream}")

Searching for bitstream files...

Found 5 bitstream file(s):
  1. bitstream/tinympcproj_N10_100Hz_float.bit
  2. bitstream/tinympcproj_N15_100Hz_float.bit
  3. bitstream/tinympcproj_N20_100Hz_float.bit
  4. bitstream/tinympcproj_N30_100Hz_float.bit
  5. bitstream/tinympcproj_N5_100Hz_float.bit

Default selected bitstream: bitstream/tinympcproj_N10_100Hz_float.bit


In [21]:
# Test all bitstreams with max_iter = 10, 100, 1000
all_test_results = []
test_maxiter_values = [10, 100, 1000]

if len(bitstreams) > 0:
    print(f"\nTesting {len(bitstreams)} bitstream(s) with max_iter values: {test_maxiter_values}")
    print("=" * 80)
    
    for idx, bitstream in enumerate(bitstreams):
        print(f"\n[{idx+1}/{len(bitstreams)}] Testing: {bitstream.name}")
        print("-" * 40)
        
        try:
            # Run performance test with warmup
            results = test_bitstream_performance(
                bitstream, 
                test_maxiter_values=test_maxiter_values,
                num_trials=10  # 10 trials after warmup for each max_iter
            )
            
            all_test_results.append(results)
            
            # Display results immediately
            print(f"  N parameter: {results['N']}")
            print(f"  Average execution times (ms):")
            for max_iter in test_maxiter_values:
                if max_iter in results['avg_times']:
                    avg_time = results['avg_times'][max_iter]
                    std_time = np.std(results['all_times'][max_iter])
                    print(f"    max_iter={max_iter:4d}: {avg_time:8.3f} ± {std_time:.3f} ms")
                    
        except Exception as e:
            print(f"  ERROR: Failed to test bitstream - {e}")
    
    print("\n" + "=" * 80)
    print("TESTING COMPLETE")
    print("=" * 80)
    
else:
    print("No bitstreams found to test.")

# Create summary table of all results
if len(all_test_results) > 0:
    # Build DataFrame for summary
    summary_data = []
    for result in all_test_results:
        row = {
            'Bitstream': Path(result['bitstream']).name,
            'N': result['N'],
        }
        # Add average times for each max_iter value
        for max_iter in test_maxiter_values:
            if max_iter in result['avg_times']:
                row[f'max_iter={max_iter} (ms)'] = f"{result['avg_times'][max_iter]:.3f}"
            else:
                row[f'max_iter={max_iter} (ms)'] = "N/A"
        summary_data.append(row)
    
    df_summary = pd.DataFrame(summary_data)
    
    print("\nSUMMARY TABLE - Average Execution Times")
    print("=" * 80)
    print(df_summary.to_string(index=False))
    print("=" * 80)
    
    # Export to CSV
    csv_filename = 'bitstream_performance_summary.csv'
    df_summary.to_csv(csv_filename, index=False)
    print(f"\nResults exported to: {csv_filename}")


Testing 5 bitstream(s) with max_iter values: [10, 100, 1000]

[1/5] Testing: tinympcproj_N10_100Hz_float.bit
----------------------------------------
Loading overlay from: bitstream/tinympcproj_N10_100Hz_float.bit
Setting FCLK0 frequency from 99.999 MHz to 250 MHz
Clock frequency set to 249.9975 MHz
Found TinyMPC IP core: tinympc_solver_0
Allocated 324 float32 memory buffer at 0x11f77000
IP core initialized with memory address: 0x0000000011f77000
  Lower 32 bits: 0x11f77000
  Upper 32 bits: 0x00000000
TinyMPC driver initialized successfully
Parameters written: max_iter=10, check_termination_iter=10
Before start - Control: 0x00000004 (ready: False, idle: True)
IP started...
IP completed after 1 polls
Hardware execution completed in 0.0023 seconds
Parameters written: max_iter=10, check_termination_iter=10
Before start - Control: 0x00000004 (ready: False, idle: True)
IP started...
IP completed after 1 polls
Hardware execution completed in 0.0015 seconds
Parameters written: max_iter=10, c

## Bitstream Switch Time Test

In [ ]:
# Linear fitting test: Execution time vs max_iter (10-200)
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

def test_linear_performance(bitstream_path, maxiter_range, num_trials=5):
    """
    Test bitstream performance across a range of maxiter values and fit linear model
    
    Args:
        bitstream_path: Path to bitstream file
        maxiter_range: Range of maxiter values to test
        num_trials: Number of trials per maxiter value
    
    Returns:
        dict: Results with linear fit parameters
    """
    # Extract N from bitstream filename
    N = extract_N_from_bitstream(bitstream_path)
    
    # Generate test data
    nx = 12
    nu = 4
    x0_test, xref_test, uref_test = generate_test_data(nx, nu, N)
    
    # Initialize hardware solver
    hw_solver = tinympc_hw(bitstream_path=str(bitstream_path))
    
    # Store results
    maxiter_values = []
    avg_times = []
    
    print(f"  Testing maxiter values: {maxiter_range[0]} to {maxiter_range[-1]} (step={maxiter_range[1]-maxiter_range[0]})")
    
    for max_iter in maxiter_range:
        # Set solver parameters
        check_termination = max_iter
        hw_solver.setup(max_iter=max_iter, check_termination=check_termination, verbose=0)
        
        # Warmup run
        hw_solver.set_x0(x0_test)
        hw_solver.set_x_ref(xref_test)
        hw_solver.set_u_ref(uref_test)
        hw_solver.solve(timeout=1.0)
        
        # Actual timing runs
        times = []
        for trial in range(num_trials):
            hw_solver.set_x0(x0_test)
            hw_solver.set_x_ref(xref_test)
            hw_solver.set_u_ref(uref_test)
            
            success = hw_solver.solve(timeout=1.0)
            
            if success:
                # Use the solver's internal timing (already in ms)
                exec_time = hw_solver.solve_time
                times.append(exec_time)
        
        if len(times) > 0:
            maxiter_values.append(max_iter)
            avg_times.append(np.mean(times))
    
    # Cleanup
    hw_solver.cleanup()
    
    # Perform linear regression
    if len(maxiter_values) > 1:
        slope, intercept, r_value, p_value, std_err = stats.linregress(maxiter_values, avg_times)
        
        return {
            'bitstream': str(bitstream_path),
            'N': N,
            'maxiter_values': maxiter_values,
            'avg_times': avg_times,
            'slope_a': slope,
            'intercept_b': intercept,
            'r_squared': r_value**2,
            'std_err': std_err
        }
    else:
        return None

# Test all bitstreams with linear fitting
print("\n" + "="*80)
print("LINEAR FITTING: EXECUTION TIME vs MAX_ITER")
print("="*80)

# Define maxiter range: 10 to 200 with step of 10
maxiter_test_range = range(10, 210, 10)

linear_fit_results = []

if len(bitstreams) > 0:
    print(f"\nTesting {len(bitstreams)} bitstream(s) with maxiter range: 10-200")
    print("-"*80)
    
    for idx, bitstream in enumerate(bitstreams):
        print(f"\n[{idx+1}/{len(bitstreams)}] Testing: {bitstream.name}")
        
        try:
            # Run linear fitting test
            result = test_linear_performance(
                bitstream,
                maxiter_test_range,
                num_trials=5  # 5 trials per maxiter value for averaging
            )
            
            if result:
                linear_fit_results.append(result)
                
                # Display fitting results
                print(f"  N parameter: {result['N']}")
                print(f"  Linear fit: t = {result['slope_a']:.6f} * maxiter + {result['intercept_b']:.3f}")
                print(f"  R-squared: {result['r_squared']:.4f}")
                print(f"  Standard error: {result['std_err']:.6f}")
                
        except Exception as e:
            print(f"  ERROR: Failed to test bitstream - {e}")
    
    print("\n" + "="*80)
    print("LINEAR FITTING COMPLETE")
    print("="*80)

# Create summary table of linear fit parameters
if len(linear_fit_results) > 0:
    print("\nLINEAR FIT PARAMETERS SUMMARY")
    print("="*80)
    print(f"{'Bitstream':<40} {'N':>3} {'a (slope)':>12} {'b (intercept)':>12} {'R²':>8}")
    print("-"*80)
    
    for result in linear_fit_results:
        bitstream_name = Path(result['bitstream']).name
        print(f"{bitstream_name:<40} {result['N']:>3} {result['slope_a']:>12.6f} {result['intercept_b']:>12.3f} {result['r_squared']:>8.4f}")
    
    print("="*80)
    
    # Export linear fit parameters to CSV
    linear_fit_data = []
    for result in linear_fit_results:
        linear_fit_data.append({
            'Bitstream': Path(result['bitstream']).name,
            'N': result['N'],
            'a_slope': result['slope_a'],
            'b_intercept': result['intercept_b'],
            'R_squared': result['r_squared'],
            'std_error': result['std_err']
        })
    
    df_linear = pd.DataFrame(linear_fit_data)
    csv_filename = 'linear_fit_parameters.csv'
    df_linear.to_csv(csv_filename, index=False)
    print(f"\nLinear fit parameters exported to: {csv_filename}")
else:
    print("No linear fitting results to display.")

In [ ]:
# Visualize linear fits
if len(linear_fit_results) > 0:
    # Create figure with subplots
    num_results = len(linear_fit_results)
    cols = min(3, num_results)
    rows = (num_results + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(6*cols, 5*rows))
    if num_results == 1:
        axes = [axes]
    elif rows == 1:
        axes = axes
    else:
        axes = axes.flatten()
    
    for idx, result in enumerate(linear_fit_results):
        ax = axes[idx] if num_results > 1 else axes[0]
        
        # Plot actual data points
        ax.scatter(result['maxiter_values'], result['avg_times'], 
                  alpha=0.6, label='Measured', s=30)
        
        # Plot fitted line
        x_fit = np.array(result['maxiter_values'])
        y_fit = result['slope_a'] * x_fit + result['intercept_b']
        ax.plot(x_fit, y_fit, 'r-', 
               label=f't = {result["slope_a"]:.4f}*maxiter + {result["intercept_b"]:.2f}', 
               linewidth=2)
        
        # Labels and title
        bitstream_name = Path(result['bitstream']).name
        ax.set_xlabel('Max Iterations', fontsize=10)
        ax.set_ylabel('Execution Time (ms)', fontsize=10)
        ax.set_title(f'{bitstream_name}\nN={result["N"]}, R²={result["r_squared"]:.4f}', 
                    fontsize=11)
        ax.legend(fontsize=9)
        ax.grid(True, alpha=0.3)
    
    # Hide extra subplots if any
    for idx in range(num_results, len(axes)):
        axes[idx].set_visible(False)
    
    plt.suptitle('Linear Fit: Execution Time vs Max Iterations', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.savefig('linear_fit_plots.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\nPlots saved to: linear_fit_plots.png")
    
    # Also create a combined plot showing all fits on one graph
    plt.figure(figsize=(10, 6))
    
    for result in linear_fit_results:
        bitstream_name = Path(result['bitstream']).name
        
        # Plot actual data points
        plt.scatter(result['maxiter_values'], result['avg_times'], 
                   alpha=0.5, s=20, label=f'{bitstream_name} (data)')
        
        # Plot fitted line
        x_fit = np.array(result['maxiter_values'])
        y_fit = result['slope_a'] * x_fit + result['intercept_b']
        plt.plot(x_fit, y_fit, '-', linewidth=2, 
                label=f'{bitstream_name} (fit, N={result["N"]})')
    
    plt.xlabel('Max Iterations', fontsize=12)
    plt.ylabel('Execution Time (ms)', fontsize=12)
    plt.title('All Bitstreams: Linear Fit Comparison', fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('linear_fit_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("Comparison plot saved to: linear_fit_comparison.png")

In [ ]:
import time
all_bitstreams = list(Path("bitstream").glob("*.bit"))

# Test bitstream switch time (initialization time only)
print("\n" + "="*60)
print("BITSTREAM SWITCH TIME TEST")
print("="*60)

if len(all_bitstreams) < 2:
    print("Need at least 2 bitstreams to test switching time")
else:
    # Initialize results storage
    switch_times = []
    
    # Number of switches to test
    num_tests = 10
    
    print(f"Testing {num_tests} bitstream initialization times...")
    print("Note: This measures the time to load and initialize a bitstream")
    
    # Alternate between bitstreams
    for i in range(num_tests):
        bitstream = all_bitstreams[i % len(all_bitstreams)]
        
        # Record start time for initialization
        start_time = time.time()
        new_solver = tinympc_hw(bitstream_path=str(bitstream))
        switch_time = (time.time() - start_time) * 1000  # Convert to ms
        switch_times.append(switch_time)
        print(f"Init {i+1} ({bitstream.name}): {switch_time:.2f} ms")
    
    # Calculate statistics
    avg_switch = np.mean(switch_times)
    std_switch = np.std(switch_times)
    
    print("\nBitstream Initialization Time Results:")
    print(f"Average initialization time: {avg_switch:.2f} ms")
    print(f"Standard deviation: {std_switch:.2f} ms")
    print("="*60)

In [ ]:
import time
all_bitstreams = list(Path("bitstream").glob("*.bit"))

# Test bitstream switch time
print("\n" + "="*60)
print("BITSTREAM SWITCH TIME TEST")
print("="*60)

if len(all_bitstreams) < 2:
    print("Need at least 2 bitstreams to test switching time")
else:
    # Initialize results storage
    switch_times = []
    
    # Number of switches to test
    num_tests = 10
    
    print(f"Testing {num_tests} switches between bitstreams...")
    
    # Alternate between first two bitstreams
    for i in range(num_tests):
        bitstream = all_bitstreams[i % len(all_bitstreams)]
        
        # Record start time
        start_time = time.time()
        new_solver = tinympc_hw(bitstream_path=str(bitstream))
        switch_time = (time.time() - start_time) * 1000  # Convert to ms
        switch_times.append(switch_time)
        print(f"Switch {i+1}: {switch_time:.2f} ms")
    
    # Calculate statistics
    avg_switch = np.mean(switch_times)
    std_switch = np.std(switch_times)
    
    print("\nResults:")
    print(f"Average switch time: {avg_switch:.2f} ms")
    print(f"Standard deviation: {std_switch:.2f} ms")
    print("="*60)
